In [1]:
import sys
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

sys.path.append('../Scripts')
from Data_Processing import DataProcessing

from tensorflow import keras
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras import backend as K
from datetime import datetime
from sklearn.preprocessing import PowerTransformer

import joblib

In [2]:
ColumnTransformer = joblib.load('../Models/Column_Transformer.pkl')
PowerTransformer = joblib.load('../Models/Power_Transformer.pkl')
ColumnTransformer_NN = joblib.load('../Models/Column_Transformer_NN.pkl')

C:\Users\mcand\Anaconda3\lib\site-packages\sklearn\base.py:324: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\mcand\Anaconda3\lib\site-packages\sklearn\base.py:324: UserWarning: Trying to unpickle estimator PowerTransformer from version 0.24.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
df = DataProcessing('../Data/train.csv')

In [3]:
X = df.drop(columns=['Lap_Time', 'Location', 'Lap_Number', 'Lap_Improvement'])

num_columns = X.select_dtypes(include='number').columns

In [4]:
#X[num_columns] = PowerTransformer.transform(X[num_columns])
trans_X = ColumnTransformer.transform(X)
trans_X_nn = ColumnTransformer_NN.transform(X)

trans_X = trans_X.toarray()
#trans_X = trans_X[:,[0, 2, 4, 6, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 72, 73]]
trans_X_nn = trans_X_nn.toarray()

In [5]:
def root_mean_squared_log_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(K.log(1+y_pred) - K.log(1+y_true))))

In [6]:
#Neural Network
nn_model = load_model('../Models/NN_model.h5', custom_objects={'root_mean_squared_log_error': root_mean_squared_log_error})

In [7]:
#Random Forest
rf_model = joblib.load('../Models/RF_Model.h5')

In [8]:
#Gradient Boost
gb_model = joblib.load('../Models/Gradient_Boost_Model.h5')

In [9]:
y_predicted_nn = nn_model.predict(trans_X_nn).reshape(-1)
y_predicted_rf = rf_model.predict(trans_X)
y_predicted_gb = gb_model.predict(trans_X)

In [10]:
submission = pd.DataFrame()
submission['NN'] = y_predicted_nn
submission['RF'] = y_predicted_rf
submission['GB'] = y_predicted_gb
submission['LAP_TIME'] = (submission['NN'] + submission['RF'] + submission['GB']) / 3
submission.drop(columns=['NN', 'RF', 'GB'], inplace=True)
submission

,LAP_TIME
0,90.046957
1,89.644665
2,84.732301
3,92.157173
4,91.692036
...,...
415,75.692009
416,73.992233
417,76.245821
418,74.900700


In [11]:
today = datetime.today().strftime('%m-%d-%y %H-%M')
submission.to_csv(f'../Submissions/Dare_In_Reality {today}.csv', index=False)

### Just Neural Network

In [ ]:
submission = pd.DataFrame()
submission['LAP_TIME'] = y_predicted_nn
submission

In [ ]:
submission.to_csv(f'../Submissions/Dare_In_Reality NN Only.csv', index=False)